In [14]:
import json
import csv
from collections import defaultdict
from sklearn.metrics import roc_auc_score, precision_score, recall_score

def count_shingle_occurrences(filename):
    shingle_bad_counts = defaultdict(int)
    shingle_total_counts = defaultdict(int)
    total_texts = 0
    bad_texts = 0

    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            if line.strip():
                try:
                    data = json.loads(line)
                    total_texts += 1
                    is_bad = not data['target']
                    if is_bad:
                        bad_texts += 1

                    unique_shingles = set(data['shingles'])

                    for shingle in unique_shingles:
                        shingle_total_counts[shingle] += 1
                        if is_bad:
                            shingle_bad_counts[shingle] += 1

                except json.JSONDecodeError as e:
                    print(f"JSON decoding error: {e}")

    return shingle_bad_counts, shingle_total_counts, total_texts, bad_texts

def classify_shingles_and_evaluate(filename, shingle_bad_counts, shingle_total_counts, total_texts, bad_texts):
    correct_predictions = 0
    total_predictions = 0
    y_true = []
    y_pred = []
    y_scores = []

    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            if line.strip():
                try:
                    data = json.loads(line)
                    total_predictions += 1

                    # Calculate phishing likelihood for the text
                    phishing_likelihood = sum(
                        shingle_bad_counts[shingle] / shingle_total_counts[shingle]
                        for shingle in data['shingles']
                        if shingle in shingle_total_counts
                    ) / len(data['shingles'])

                    # Classification threshold is based on the rate of bad texts
                    threshold = bad_texts / total_texts
                    prediction = 1 if phishing_likelihood > threshold else 0
                    actual = 1 if not data['target'] else 0

                    y_true.append(actual)
                    y_pred.append(prediction)
                    y_scores.append(phishing_likelihood)

                    if prediction == actual:
                        correct_predictions += 1

                except json.JSONDecodeError as e:
                    print(f"JSON decoding error: {e}")

    # Calculate accuracy, precision, recall, and ROC AUC
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_scores)

    print(f"Accuracy: {accuracy:.2%}")
    print(f"Precision: {precision:.2%}")
    print(f"Recall: {recall:.2%}")
    print(f"ROC AUC: {roc_auc:.2%}")

def save_to_csv(shingle_bad_counts, shingle_total_counts, output_filename):
    with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Shingle', 'Count in phishing texts', 'Count in all texts'])

        all_shingles = set(shingle_total_counts.keys()).union(shingle_bad_counts.keys())
        for shingle in all_shingles:
            writer.writerow([
                shingle,
                shingle_bad_counts.get(shingle, 0),
                shingle_total_counts.get(shingle, 0)
            ])

filename = '/content/dataset_shingles_maria_sample.csv'
shingle_bad_counts, shingle_total_counts, total_texts, bad_texts = count_shingle_occurrences(filename)

output_filename = 'shingle_counts.csv'
save_to_csv(shingle_bad_counts, shingle_total_counts, output_filename)

classify_shingles_and_evaluate(filename, shingle_bad_counts, shingle_total_counts, total_texts, bad_texts)


JSON decoding error: Unterminated string starting at: line 1 column 106523 (char 106522)
JSON decoding error: Unterminated string starting at: line 1 column 106523 (char 106522)
Accuracy: 99.24%
Precision: 99.98%
Recall: 99.16%
ROC AUC: 99.99%
